<a href="https://colab.research.google.com/github/jjjuurang/CLIP/blob/main/FSD50KZSmAPSK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install timm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 512 kB 7.8 MB/s 


In [2]:
!pip install --upgrade torch==1.9.1+cu111 torchvision==0.10.1+cu111 -f https://download.pytorch.org/whl/torch_stable.html
#!pip install --upgrade https://download.pytorch.org/whl/nightly/cu111/torch-1.11.0.dev20211012%2Bcu111-cp37-cp37m-linux_x86_64.whl https://download.pytorch.org/whl/nightly/cu111/torchvision-0.12.0.dev20211012%2Bcu111-cp37-cp37m-linux_x86_64.whl
!git clone https://github.com/openai/CLIP
!pip install -e ./CLIP
!pip install einops ninja
import sys
sys.path.append('./CLIP')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |█████████████                   | 834.1 MB 7.0 MB/s eta 0:02:53tcmalloc: large alloc 1147494400 bytes == 0x38f16000 @  0x7f59c7573615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x593dd7
     |████████████████▌               | 1055.7 MB 1.5 MB/s eta 0:11:02tcmalloc: large alloc 1434370048 bytes == 0x7d56c000 @  0x7f59c7573615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x593dd7
     |█████

In [3]:
import torch 
import timm
import librosa
import cv2
import numpy as np
import clip
from collections import OrderedDict

class AudioEncoder(torch.nn.Module):
    def __init__(self, backbone_name="resnet18"):
        super(AudioEncoder, self).__init__()
        self.backbone_name = backbone_name
        self.conv = torch.nn.Conv2d(1, 3, (3, 3))
        self.feature_extractor = timm.create_model(self.backbone_name, num_classes=512, pretrained=True)

    def forward(self, x):
        x = self.conv(x)
        x = self.feature_extractor(x)
        return x

/usr/local/lib/python3.7/dist-packages/resampy/interpn.py:114: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
  _resample_loop_p(x, t_out, interp_win, interp_delta, num_table, scale, y)


In [4]:
device = torch.device('cuda:0')
print('Using device:', device, file=sys.stderr)

Using device: cuda:0


In [5]:
clip_model, _ = clip.load("ViT-B/32", device=device)

100%|████████████████████████████████████████| 338M/338M [00:02<00:00, 142MiB/s]


In [6]:
def copyStateDict(state_dict):
    if list(state_dict.keys())[0].startswith("module"):
        start_idx = 1
    else:
        start_idx = 0
    new_state_dict = OrderedDict()
    for k, v in state_dict.items():
        name = ".".join(k.split(".")[start_idx:])
        new_state_dict[name] = v
    return new_state_dict

In [7]:
audio_encoder = AudioEncoder()
audio_encoder.load_state_dict(copyStateDict(torch.load("/content/resnet18.pth", map_location=device)))

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth


<All keys matched successfully>

In [8]:
import pandas as pd
csv = pd.read_csv("/content/vocabulary.csv",names=["index","labels","third"])

In [9]:
print(len(csv['labels'].values.tolist()))

200


In [10]:
all = csv['labels'].values.tolist()
print(all)

['Accelerating_and_revving_and_vroom', 'Accordion', 'Acoustic_guitar', 'Aircraft', 'Alarm', 'Animal', 'Applause', 'Bark', 'Bass_drum', 'Bass_guitar', 'Bathtub_(filling_or_washing)', 'Bell', 'Bicycle', 'Bicycle_bell', 'Bird', 'Bird_vocalization_and_bird_call_and_bird_song', 'Boat_and_Water_vehicle', 'Boiling', 'Boom', 'Bowed_string_instrument', 'Brass_instrument', 'Breathing', 'Burping_and_eructation', 'Bus', 'Buzz', 'Camera', 'Car', 'Car_passing_by', 'Cat', 'Chatter', 'Cheering', 'Chewing_and_mastication', 'Chicken_and_rooster', 'Child_speech_and_kid_speaking', 'Chime', 'Chink_and_clink', 'Chirp_and_tweet', 'Chuckle_and_chortle', 'Church_bell', 'Clapping', 'Clock', 'Coin_(dropping)', 'Computer_keyboard', 'Conversation', 'Cough', 'Cowbell', 'Crack', 'Crackle', 'Crash_cymbal', 'Cricket', 'Crow', 'Crowd', 'Crumpling_and_crinkling', 'Crushing', 'Crying_and_sobbing', 'Cupboard_open_or_close', 'Cutlery_and_silverware', 'Cymbal', 'Dishes_and_pots_and_pans', 'Dog', 'Domestic_animals_and_pets',

In [11]:
text_inputs = torch.cat([clip.tokenize(l) for l in csv['labels'].values.tolist()]).to(device)

In [12]:
import pandas as pd
csvev = pd.read_csv("/content/s5.csv",names=["indexeev","labelsev","thirdev"])

FileNotFoundError: ignored

In [ ]:
gtlabel = csvev['labelsev'].values.tolist()

In [ ]:
gtindex = csvev["indexeev"].values.tolist()

In [ ]:
import os
path_dir = "/content/sound5/"
file_list = os.listdir(path_dir)

In [ ]:
import natsort

In [ ]:
print(natsort.natsorted(file_list))

In [ ]:
gtt = torch.zeros(len(file_list),200)

In [ ]:
for i in range(len(file_list)):
  gt = gtlabel[i].split(",")
  for g in gt:
    gtt[i,all.index(g)] = 1

In [ ]:
with torch.no_grad():
    text_embedding = clip_model.encode_text(text_inputs).float()


In [ ]:
text_embedding = text_embedding / text_embedding.norm(dim=-1, keepdim=True)

In [ ]:
sts = torch.zeros(len(file_list),200)

In [ ]:
print(file_list)

In [131]:
for sn in range(len(file_list)):
  n_mels = 128
  time_length = 864
  resize_resolution = 512
  y, sr = librosa.load("/content/sound5/"+ file_list[sn], sr=44100)
  audio_inputs = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=n_mels)
  audio_inputs = librosa.power_to_db(audio_inputs, ref=np.max) / 80.0 + 1

  zero = np.zeros((n_mels, time_length))
  h, w = audio_inputs.shape
  if w >= time_length:
      j = (w - time_length) // 2
      audio_inputs = audio_inputs[:,j:j+time_length]
  else:
      j = (time_length - w) // 2
      zero[:,:w] = audio_inputs[:,:w]
      audio_inputs = zero
    
  audio_inputs = cv2.resize(audio_inputs, (n_mels, resize_resolution))
  audio_inputs = np.array([audio_inputs])
  audio_inputs = torch.from_numpy(audio_inputs.reshape((1, 1, n_mels, resize_resolution))).float()
  with torch.no_grad():
    audio_embedding = audio_encoder(audio_inputs).cuda()
  audio_embedding = audio_embedding / audio_embedding.norm(dim=-1, keepdim=True)
  sts[sn,:] = (100.0 * audio_embedding @ text_embedding.T).sigmoid()
  stc[sn,:] = (100.0 * audio_embedding @ text_embedding.T).sigmoid()


In [146]:
pre = torch.zeros(len(file_list),200)

In [147]:
for i in range(len(file_list)):
  values, indices = sts[i,:].topk(200)
  for value, index in zip(values, indices):
    if value.item() > 0.03:
      pre[i,index] = 1



In [163]:
gtt

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0

In [164]:
pre

tensor([[0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0.,
         0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
         0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
         0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0